A: Building DNN with 20 layers, 100 neurons each. Use He initialization and ELU activation function.

B: Train the network on the CIFAR10 dataset.
<br>
keras.datasets.cifar10.load_data()
<br>
Dataset is 60k 32 by 32 pixel color images with 10 classes. Use 50k for training and 10k for testing.
<br> 
Use Nadam optimization and Early Stopping. Output layer should have 10 neurons and use softmax.

In [ ]:
# Import TensorFlow and the dataset
import tensorflow as tf
from tensorflow import keras

cifar = keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
# Dataset properties
print(X_train_full.shape)
print(X_train_full.dtype)

(50000, 32, 32, 3)
uint8


In [ ]:
# Class names
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# Build classification MLP with twenty hidden layers, 100 neurons each

# Create model with Flatten layer, then add 20 dense layers in the next block.
model = keras.models.Sequential([ 
                                 keras.layers.Flatten()
])

In [ ]:
# Loop to add the 20 hidden layers. Adds BatchNormalization, Dropout, and Dense layers.
x = 0
while x < 20 : 
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
    x += 1

# Add output layer using softmax
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
# Nadam optimizer
nadam = keras.optimizers.Nadam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'
)

early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# Compile with the optimizer (remove nadam variable, it has one pre-built)
model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) #u can choose the parameters but they're the defaults

In [ ]:
X_train_full = X_train_full.reshape(-1, 32*32*3)

X_train_full = X_train_full/255.0


In [ ]:
# Train model
history = model.fit(X_train_full, y_train_full, epochs=30)

Epoch 1/30
1563/1563 [==============================] - 32s 20ms/step - loss: 2.2256 - accuracy: 0.1757
Epoch 2/30
1563/1563 [==============================] - 33s 21ms/step - loss: 1.9705 - accuracy: 0.2598
Epoch 3/30
1563/1563 [==============================] - 31s 20ms/step - loss: 1.8941 - accuracy: 0.2940
Epoch 4/30
1563/1563 [==============================] - 32s 20ms/step - loss: 1.8440 - accuracy: 0.3216
Epoch 5/30
1563/1563 [==============================] - 36s 23ms/step - loss: 1.8099 - accuracy: 0.3415
Epoch 6/30
1563/1563 [==============================] - 31s 20ms/step - loss: 1.7879 - accuracy: 0.3551
Epoch 7/30
1563/1563 [==============================] - 32s 20ms/step - loss: 1.7664 - accuracy: 0.3669
Epoch 8/30
1563/1563 [==============================] - 32s 20ms/step - loss: 1.7399 - accuracy: 0.3795
Epoch 9/30
1563/1563 [==============================] - 32s 20ms/step - loss: 1.7253 - accuracy: 0.3847
Epoch 10/30
1563/1563 [==============================] - 32s 20m

In [ ]:
# GridSearchCV for learning rate optimization
# Used tutorial from: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# to combat scikit-learn being unable to clone the model object because of it missing the scikit-learn standard get_params method.
from sklearn.model_selection import GridSearchCV
# learning rate 2 magnitudes up and down from default value. need to remove default value 
param_dist = [
             {'learning_rate': [0.00001, 0.0001, 0.001, 0.01, 0.1]}
]
clf = GridSearchCV(model, param_dist, cv=5, n_jobs=-1, scoring='accuracy', verbose=1)
# Pass in the **fit_params at the end to be passed to the classifiers' fit method
clf.fit(X_train_full, y_train_full)


print("Best Estimator: ", clf.best_estimator__)
print("Best Params: ", clf.best_params__)
print("Best Score: ", clf.best_score__)

TypeError: ignored

TensorBoard for viewing

In [ ]:
import os
root_logdir = os.path.join(os.curdir, "logs")

def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
#u need to compile the model b4 fitting


In [ ]:
# TensorBoard viewer thing
%load_ext tensorboard
%tensorboard --logdir=./logs --port=6006

In [ ]:
# Reload TensorBoard with changes (commented out because previous step not solved)
# %load_ext tensorboard
# %tensorboard --logdir=./logs --port=6006